# Preprocessed Citibike Data: Exploratory Data Analysis

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas.api.types import CategoricalDtype
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Import and load data

Import preprocessed data:

In [2]:
cb_raw = pd.read_csv('../../data/02_processed/concat_file_processed.csv', parse_dates=True)

Drop unused index:

In [3]:
cb_raw = cb_raw.drop('Unnamed: 0', axis=1)

Make a copy: 

In [4]:
cb = cb_raw.copy()

## Inspect data frame

Get dimensions of data frame and datatypes of each feature:

In [5]:
cb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4605356 entries, 0 to 4605355
Data columns (total 15 columns):
bikeid                   int64
birthyear                float64
endstationid             float64
endstationlatitude       float64
endstationlongitude      float64
endstationname           object
gender                   int64
startstationid           float64
startstationlatitude     float64
startstationlongitude    float64
startstationname         object
starttime                object
stoptime                 object
tripduration             float64
usertype                 object
dtypes: float64(8), int64(2), object(5)
memory usage: 527.0+ MB


Randomly sample five observations:

In [6]:
cb.sample(n=5)

,bikeid,birthyear,endstationid,endstationlatitude,endstationlongitude,endstationname,gender,startstationid,startstationlatitude,startstationlongitude,startstationname,starttime,stoptime,tripduration,usertype
2062405,15719,1978.0,445.0,40.727408,-73.981420,E 10 St & Avenue A,1,252.0,40.732264,-73.998522,MacDougal St & Washington Sq,2016-04-22 03:38:10,2016-04-22 03:50:22,732.0,Subscriber
514438,17454,1993.0,459.0,40.746745,-74.007756,W 20 St & 11 Ave,1,453.0,40.744751,-73.999154,W 22 St & 8 Ave,2015-04-27 18:55:30,2015-04-27 19:00:10,279.0,Subscriber
1480836,17904,1979.0,3093.0,40.717452,-73.958509,N 6 St & Bedford Ave,1,3107.0,40.723117,-73.952123,Bedford Ave & Nassau Ave,2015-08-28 18:00:32,2015-08-28 18:05:01,269.0,Subscriber
74440,16119,1980.0,529.0,40.757570,-73.990985,W 42 St & 8 Ave,1,2003.0,40.733812,-73.980544,1 Ave & E 18 St,2017-09-25 16:20:41,2017-09-25 16:44:20,1418.0,Subscriber
49983,28871,1975.0,486.0,40.746201,-73.988557,Broadway & W 29 St,1,251.0,40.723180,-73.994800,Mott St & Prince St,2017-09-17 20:59:19,2017-09-17 21:11:03,704.0,Subscriber


Check for missing values – there should be none:

In [7]:
cb.isnull().sum()

bikeid                   0
birthyear                0
endstationid             0
endstationlatitude       0
endstationlongitude      0
endstationname           0
gender                   0
startstationid           0
startstationlatitude     0
startstationlongitude    0
startstationname         0
starttime                0
stoptime                 0
tripduration             0
usertype                 0
dtype: int64

## Data preprocessing

Show unique values per feature:

In [8]:
pd.DataFrame.from_records([(col, cb[col].nunique()) for col in cb.columns],
                          columns=['Feature', 'Number of Unique Values']).sort_values(by=['Number of Unique Values'])

,Feature,Number of Unique Values
6,gender,3
14,usertype,3
1,birthyear,85
7,startstationid,1058
2,endstationid,1073
10,startstationname,1089
5,endstationname,1107
9,startstationlongitude,1121
4,endstationlongitude,1140
8,startstationlatitude,1201


### Define features

Define numerical features:

In [9]:
map_feat = ['endstationlatitude','endstationlongitude','startstationlatitude','startstationlongitude']
datetime_feat = ['birthyear','starttime', 'stoptime']
cont_feat = map_feat + datetime_feat + ['tripduration']

Define categorical features:

In [10]:
num_nom_feat = ['bikeid','endstationid','startstationid','gender']
cat_nom_feat = ['endstationname','startstationname','usertype']
nom_feat = num_nom_feat + cat_nom_feat

### Perform datatype conversions

Convert to datetime format:

In [11]:
cb['starttime'] = pd.to_datetime(cb.starttime)
cb['stoptime'] = pd.to_datetime(cb.stoptime)
cb['birthyear'] = pd.to_datetime(cb.birthyear, format='%Y')

In [12]:
cb['birthyear'] = cb['birthyear'].dt.year

Convert unique ID numbers and trip durations to `int64`:

In [13]:
float_to_int = ['startstationid','endstationid','tripduration']

cb[float_to_int] = cb[float_to_int].apply(lambda x: x.astype('int64'))

Convert untreated nominal features to categorical datatype:

In [14]:
cb[cat_nom_feat] = cb[cat_nom_feat].apply(lambda x: x.astype('category'))

Check for successful datatype conversion and reduction in memory:

In [15]:
cb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4605356 entries, 0 to 4605355
Data columns (total 15 columns):
bikeid                   int64
birthyear                int64
endstationid             int64
endstationlatitude       float64
endstationlongitude      float64
endstationname           category
gender                   int64
startstationid           int64
startstationlatitude     float64
startstationlongitude    float64
startstationname         category
starttime                datetime64[ns]
stoptime                 datetime64[ns]
tripduration             int64
usertype                 category
dtypes: category(3), datetime64[ns](2), float64(4), int64(6)
memory usage: 443.7 MB


## Feature engineering

Define helper function to calculate distance between coordinates:

In [34]:
from pyproj import Geod

def calc_vincenty_dist(lat1, lon1, lat2, lon2): 
    az12, az21, dist = Geod(ellps='WGS84').inv(lon1, lat1, lon2, lat2)
    return dist

In [23]:
# # Cartesian Coordinate system, output in latlong dist
# def calc_manh_distc(lat, lon):
#     return sum(abs(lat_i-lat_j) for lat_i, lat_j in zip(lat, lon))

In [53]:
# Haversine & inclination 29 degrees to True north
# Credits: https://www.movable-type.co.uk/scripts/latlong.html,
# https://gist.github.com/jkAtGitHub/8ae7da4d5dacb9969bff43500b5efbc0#file-manhattan_dist-py
def calc_haversine_dist(lat1, lon1, lat2, lon2):
    lat1, lat2, lon1, lon2 = np.radians(lat1), np.radians(lat2), np.radians(lon1), np.radians(lon2)
    dlat, dlon = lat2 - lat1, lon2 - lon1
    r = 3963 # Earth's radius in miles
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
    
    total_dist = r * c
    return total_dist

def calc_manh_disth(lat1, lon1, lat2, lon2):
    start = np.stack([lat1, lon1], axis=1)
    end = np.stack([lat2, lon2], axis=1)
    
    theta1, theta2 = np.radians(-28.904), np.radians(28.904)
    rmat1 = np.array([[np.cos(theta1), np.sin(theta1)], 
                      [-np.sin(theta1), np.cos(theta1)]])
    rmat2 = np.array([[np.cos(theta2), np.sin(theta2)], 
                      [-np.sin(theta2), np.cos(theta2)]])
    
    start_rot = rmat1 @ start.T
    end_rot = rmat1 @ end.T
    
    hinge = np.stack((start_rot[0,:], end_rot[1,:]))
    hinge_coords = rmat2 @ hinge
    
    manh_dist = calc_haversine_dist(start.T[0], start.T[1], hinge_coords[0], hinge_coords[1]) + calc_haversine_dist(hinge_coords[0], hinge_coords[1], end.T[0], end.T[1])
    return manh_dist

Create new feature, `distance` (in miles):

In [41]:
cb['tripdistance'] = calc_vincenty_dist(cb.startstationlatitude.tolist(), 
                                        cb.startstationlongitude.tolist(),
                                        cb.endstationlatitude.tolist(),
                                        cb.endstationlongitude.tolist())
cb['tripdistance'] = cb['tripdistance'].apply(lambda x: x*0.000621371) # Convert from meters to miles

In [42]:
cb['tripdistance']

0          0.860408
1          0.951472
2          2.929130
3          0.491748
4          0.379186
             ...   
4605351    2.505844
4605352    0.498789
4605353    1.033515
4605354    1.171593
4605355    0.285159
Name: tripdistance, Length: 4605356, dtype: float64

In [54]:
cb['manhdistance'] = calc_manh_disth(cb.startstationlatitude,
                                     cb.startstationlongitude,
                                     cb.endstationlatitude,
                                     cb.endstationlongitude)

In [55]:
cb['manhdistance']

0          1.072805
1          1.228247
2          3.553228
3          0.539227
4          0.514947
             ...   
4605351    3.258176
4605352    0.636966
4605353    1.624054
4605354    1.406205
4605355    0.358765
Name: manhdistance, Length: 4605356, dtype: float64

Create new features, `startmonth` and `endmonth`:

In [25]:
cb['startmonth'] = cb['starttime'].dt.month
cb['stopmonth'] = cb['stoptime'].dt.month

Create new features for days of week, `startdayofweek` and `enddayofweek` (along with their numerical codes):

In [20]:
cb['startdayofweek'] = cb['starttime'].dt.day_name()
cb['startdayofweek_code'] = cb['starttime'].dt.dayofweek
cb['stopdayofweek'] = cb['stoptime'].dt.day_name()
cb['stopdayofweek_code'] = cb['stoptime'].dt.dayofweek

In [21]:
# def trip_day(df):
#     start_temp = pd.DataFrame({'startday': df['startdayofweek'].tolist(), 'code': df['startdayofweek_code'].tolist()})
#     end_temp = pd.DataFrame({'endday': df['enddayofweek'].tolist(), 'code': df['enddayofweek_code'].tolist()})
#     return pd.merge(start_temp, end_temp, left_on='startday', right_on='endday')

# start_temp = pd.DataFrame({'day': cb['startdayofweek'].tolist(), 'code': cb['startdayofweek_code'].tolist()})
# end_temp = pd.DataFrame({'day': cb['enddayofweek'].tolist(), 'code': cb['enddayofweek_code'].tolist()})

# df_list = [start_temp, end_temp]
# for df in df_list:
#     df.set_index(['day'], inplace=True)

# df = pd.concat(df_list, axis=1) # join='inner'
# df.reset_index(inplace=True)

Create new features, `starthour` and `endhour`:

In [26]:
cb['starthour'] = cb['starttime'].dt.hour
cb['endhour'] = cb['stoptime'].dt.hour

Create new features, `startboro` and `endboro`:

In [45]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm._tqdm_notebook import tqdm_notebook

In [61]:
coords_df = pd.DataFrame({'startcoords': zip(cb.startstationlatitude, cb.startstationlongitude), 
              'endcoords': zip(cb.endstationlatitude, cb.endstationlongitude)})

In [55]:
geolocator = Nominatim(user_agent='myGeocoder')

In [118]:
loc_dict = locator.reverse(coords_df.startcoords[0]).raw

In [ ]:
coords_df.apply(lambda x: locator.reverse(x.startcoords), axis=1).suburb

In [ ]:
find_location(coords_df.startcoords)

Create new features, `startneighborhood` and `endneighborhood`:

Create new feature, `uniquetripid`:

## Inspect data